In [1]:
#imports
import spatialdata
import dask_image.imread
import geopandas as gpd
import anndata as ad

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


# Create a sdata object for testing

In [21]:
#paths
path_to_mIF = "/Volumes/Extreme SSD/P26_BCCN/P26E07_E08_E09_SOPA_Gates/processed_images/991.ome.tif"
path_to_mask = "/Volumes/Extreme SSD/P26_BCCN/P26E07_E08_E09_SOPA_Gates/expanded-masks/991.tif"
path_to_shapes = "../data/geojsons/Primary/processed/20241129_1442_Primary991_harmonized.geojson"
# path_to_imaging_table = "/Volumes/Extreme SSD/P26_BCCN/P26E07_E08_E09_SOPA_Gates/adatas/20241002_1218_991_phenotyped.h5ad"
path_to_proteomics = "../data/proteomic/20241201_170620_filtered_imputed_outerjoin_mergefirst.h5ad"

In [3]:
# initiate spatialdata object
sdata = spatialdata.SpatialData()

# load image and mask as dask arrays
image = dask_image.imread.imread(path_to_mIF)
mask_stack = dask_image.imread.imread(path_to_mask)

# mask has dimensions (c,y,x), by slicing the dask array we convert from (c,y,x) to (y,x) which is needed for Labels2DModel.parse
mask = mask_stack[0]

# parse and validate with spatialdata.models
sdata["image"] = spatialdata.models.Image2DModel.parse(image)
sdata["mask"] = spatialdata.models.Labels2DModel.parse(mask)

INFO     no axes information specified in the object, setting `dims` to: ('c', 'y', 'x')                           
INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                


In [8]:
# load polygon geojson
gdf = gpd.read_file(path_to_shapes)
# make column matching proteomic table the index
gdf = gdf.set_index("harmonized_class")
# load to sdata
sdata["Primary_contours"] = spatialdata.models.ShapesModel.parse(gdf)

In [25]:
#load proteomic table
adata = ad.read_h5ad(path_to_proteomics)

adata.obs["region"] = "Primary_contours"
adata.obs["region"] = adata.obs["region"].astype("category")

In [26]:
table = spatialdata.models.TableModel.parse(adata, region = "Primary_contours", region_key = "region", instance_key = "shape_key")
sdata["proteomics"] = table

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/spatialdata/_core/_elements.py:116: UserWarning: Key `proteomics` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


In [32]:
sdata['proteomics'] = switch_adat_var_index(sdata['proteomics'], "Genes")

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/spatialdata/_core/_elements.py:116: UserWarning: Key `proteomics` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


In [30]:
def switch_adat_var_index(adata, new_index):
    """
    Created by Jose Nimo on 2023-07-01
    Lastest modified by Jose Nimo on 2024-11-16

    Description:
    Switch the index of adata.var to a new index. Useful for switching between gene names and protein names.

    Arg:
        adata: anndata object
        new_index: pandas series, new index to switch to
    Returns:
        adata: anndata object, with the new index
    """
    adata_copy = adata.copy()

    adata_copy.var[adata_copy.var.index.name] = adata_copy.var.index
    adata_copy.var.set_index(new_index, inplace=True)
    adata_copy.var.index.name = new_index
    
    return adata_copy

In [ ]:
sdata['proteomics'].var.sort_index(inplace=True) #much easier to find a gene like this

In [ ]:
import napari_spatialdata

interactive = napari_spatialdata.Interactive(sdata)
interactive.run()

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
2024-12-01 17:17:07.257 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are n